This is a very simple example of neural network. Its purpose is to approximate an unknown single valued function using a dense deep network. 
The user is invited to play with it, modifying:
1. the hidden function
2. the number of layers and neurons per layer
3. activation functions, number of epochs, and so on.

In [3]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

"myhiddenfunction" is the definition of the function you want to approximate by means of a neural network (NN). The definition is hidden to the NN, who can only access it as a blackbox, to get training samples. 
This is implemented by means of a generator (a special kind of function), taking in input a number (batchsize) and returning a pair of input output vectors of length batchsize. Each input is a random number in the interval [-pi,pi] and the output is computed by means of myhiddenfunction.

In [26]:
def myhiddenfunction(x):
  #define your favourite function
  #output in range 0-1 if last activation is a sigmoid!
  return (np.sin(x)**3 + np.sqrt(np.cos(x)/3 + 1)/3)
  
def generator(batchsize):
    while True:
      inputs = np.random.uniform(low=-np.pi,high=np.pi,size=batchsize)
      outputs = np.zeros(batchsize)
      for i in range(0,batchsize):
          outputs[i] = myhiddenfunction(inputs[i])
      yield (inputs,outputs)

If you want to see an example of a generated batch, you need to invoke next on the generator

In [28]:
print(next(generator(5)))

(array([-0.09418854,  2.98454536,  1.49216279, -2.29262957,  1.19281995]), array([ 0.38385497,  0.27682748,  1.32842771, -0.12853686,  1.15607685]))


Now we define the network. The function we want to approximate is single valued, so the network will have a single input and a single output, and its (dense) structure is completely defined by a 
list specifying the number of neurons per layer


Here is an alternative, more modular definition of the same network.

In [34]:
# size of hidden layers (one for each of them)
inner_layers_dims = [20,30,20,30]

input_layer = Input(shape=(1,))
x = input_layer
for i in range(0,len(inner_layers_dims)):
  x = Dense(inner_layers_dims[i], activation='relu')(x)
output_layer = Dense(1, activation='relu')(x)

mymodel = Model(input_layer, output_layer)

We can now have a look at the model we just generated:

In [35]:
mymodel.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
dense_45 (Dense)             (None, 20)                40        
_________________________________________________________________
dense_46 (Dense)             (None, 30)                630       
_________________________________________________________________
dense_47 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_48 (Dense)             (None, 30)                630       
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 31        
Total params: 1,951
Trainable params: 1,951
Non-trainable params: 0
________________________________________________________

Try to be sure you correctly understand the number of learning parameters for each layer. 
For a dense layer with n input neurons and m output neurons, your have nxm weights + m biases.
For instance, for a 20-to-30 layer, we have 20x30+30 = 630 parameters.

We are finally ready to compile our model and train it.
As loss function we use mean square error (mse). 
The "optimizer" is the technique used to tune the learning rate during backpropagation: you may ignore it for the moment.

In [36]:
mymodel.compile(optimizer='adam', loss='mse')

In [37]:
batchsize = 64

mymodel.fit_generator(generator(batchsize), steps_per_epoch=1000, epochs=10)

Epoch 1/10
1000/1000 [==============================] - 12s 12ms/step - loss: 0.4264
Epoch 2/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.4221
Epoch 3/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.4217
Epoch 4/10
1000/1000 [==============================] - 10s 10ms/step - loss: 0.4218
Epoch 5/10
1000/1000 [==============================] - 12s 12ms/step - loss: 0.4243
Epoch 6/10
1000/1000 [==============================] - 12s 12ms/step - loss: 0.4216
Epoch 7/10
1000/1000 [==============================] - 12s 12ms/step - loss: 0.4254
Epoch 8/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.4228
Epoch 9/10
1000/1000 [==============================] - 10s 10ms/step - loss: 0.4241
Epoch 10/10
1000/1000 [==============================] - 12s 12ms/step - loss: 0.4212


If everything is working correctly, the loss should decrease during training.  
If it doesn't, it means that, for some reason, the network is not learning.

We are finally ready to check the result of the approximation. We plot the hidden function in red, 
and the approximation computed by the network in blu.

In [38]:
x = np.arange(-np.pi,np.pi,0.05)
y = [myhiddenfunction(a) for a in x]
z = [z[0] for z in mymodel.predict(np.array(x))]
plt.plot(x,y,'r',x,z,'b')
plt.legend(['real','predicted'], loc='upper right')
plt.show() 


Now is your turn. Modify:

1. the definition of the hidden function
2. the number of layers/neurons per layer; you just have to change inner_layers_dims in block 6.

Have fun.